In [ ]:
#import required packages

import numpy as np
import os
from nilearn.connectome import ConnectivityMeasure
from nilearn.input_data import NiftiLabelsMasker
from nilearn import plotting

#load atlas and labels

maps = '/data/netapp01/work/elena/fc_analysis_combined/ROIs/Schaefer100_FreeSurfer_dseg_tpl-MNIPediatricAsym_cohort-4_114ROIs.nii.gz'

#define subjects

subjs = [i for i in os.listdir('/data/netapp01/work/elena/fc_analysis_combined/iCATS_fmriprep_v2020/wave1/fmriprep/') 
         if i.startswith('sub-') 
         if not i.endswith('.html')]

#apply a mask to extract timeseries - i.e., ROIs in the atlas
#note that Lables is for deterministic parcellation (non overlapping volumes)

masker = NiftiLabelsMasker(labels_img=maps, #region definitions
                           standardize=True, #signal is zscored
                           memory='nilearn_cache', 
                           verbose=2, 
                           memory_level=1, 
                           high_pass=0.01,
                           t_r=2.4) 

#define preprocessed image
#go from nifti files to the signal time series and add confounders

subjects_time_series = []

for subj in subjs:
   
    time_series = masker.fit_transform('/data/netapp01/work/elena/fc_analysis_combined/iCATS_fmriprep_v2020/wave1/fmriprep/' + subj + '/func/' +  subj + '_task-rest_space-MNIPediatricAsym_cohort-4_res-2_desc-preproc_bold.nii.gz', 
                                       confounds='/data/netapp01/work/elena/fc_analysis_combined/iCATS_fmriprep_v2020/wave1/fmriprep/' + subj + '/func/' +  subj + '_task-rest_desc-confounds_timeseries_selected.txt') 
    subjects_time_series.append(time_series)

#calculate correlation matrix

correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrices = correlation_measure.fit_transform(subjects_time_series)

#save the correlation matrices - one per subject

for conmat, subj in zip(correlation_matrices, subjs):        
        np.savetxt('/data/netapp01/work/elena/fc_analysis_combined/connmatr/iCATS_Schaefer/wave1/' + subj + '_wave1.csv',   conmat,
                    delimiter=' ', fmt='%.5f', comments='')